<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

<br>

# <font color="#76b900">**Notebook 7:** Retrieval-Augmented Generation with Vector Stores</font>

<br>

In the previous notebook, we learned about embedding models and exercised some of their capabilities. We discussed their intended use cases of longer-form document comparison and found ways to use it as a backbone for more custom semantic comparisons. This notebook will progress these ideas toward the retrieval model's intended use case and explore how to build chatbot systems that rely on *vector stores* to automatically save and retrieve information.

<br>

### **Learning Objectives:**

- Understand how semantic-similarity-backed systems can facilitate easy-to-use retrieval formulations.

- Learn how to incorporate retrieval modules into your chat model systems for a retrieval-augmented generation (RAG) pipeline, which can be applied to tasks like document retrieval and conversation memory buffers.

<br>

### **Questions To Think About:**

- This notebook does not attempt to incorporate hierarchical reasoning or non-naive RAG (such as planning agents). Consider what modifications would be necessary to make these components work in an LCEL chain.

- Consider when it would be best to move your vector store solution into a scalable service and when a GPU will become necessary for optimization.

<br>

### **Notebook Source:**

- This notebook is part of a larger [**NVIDIA Deep Learning Institute**](https://www.nvidia.com/en-us/training/) course titled [**Building RAG Agents with LLMs**](https://learn.next.courses.nvidia.com/courses/course-v1:DLI+S-FX-15+V1/about). If sharing this material, please give credit and link back to the original course.

<br>


### **Environment Setup:**

In [1]:
%%capture
## ^^ Comment out if you want to see the pip install process

## Necessary for Colab, not necessary for course environment
# %pip install -q langchain langchain-nvidia-ai-endpoints gradio rich
# %pip install -q arxiv pymupdf faiss-cpu
    
## If you're in colab and encounter a typing-extensions issue,
##  restart your runtime and try again
from langchain_nvidia_ai_endpoints._common import NVEModel

In [2]:
from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

console = Console()
base_style = Style(color="#76B900", bold=True)
pprint = partial(console.print, style=base_style)

from getpass import getpass
import requests
import os

hard_reset = False  ## <-- Set to True if you want to reset your NVIDIA_API_KEY
while "nvapi-" not in os.environ.get("NVIDIA_API_KEY", "") or hard_reset:
    try: 
        response = requests.get("http://docker_router:8070/get_key").json()
        assert response.get('nvapi_key')
    except: response = {'nvapi_key' : getpass("NVIDIA API Key: ")}
    os.environ["NVIDIA_API_KEY"] = response.get("nvapi_key")
    try: requests.post("http://docker_router:8070/set_key/", json={'nvapi_key' : os.environ["NVIDIA_API_KEY"]}).json()
    except: pass
    hard_reset = False
    if "nvapi-" not in os.environ.get("NVIDIA_API_KEY", ""):
        print("[!] API key assignment failed. Make sure it starts with `nvapi-` as generated from the model pages.")

print(f"Retrieved NVIDIA_API_KEY beginning with \"{os.environ.get('NVIDIA_API_KEY')[:9]}...\"")
from langchain_nvidia_ai_endpoints._common import NVEModel
NVEModel().available_models

Retrieved NVIDIA_API_KEY beginning with "nvapi-7bN..."


{'playground_llama2_code_34b': 'df2bee43-fb69-42b9-9ee5-f4eabbeaf3a8',
 'playground_llama2_code_13b': 'f6a96af4-8bf9-4294-96d6-d71aa787612e',
 'playground_llama_guard': 'b34280ac-24e4-4081-bfaa-501e9ee16b6f',
 'playground_sdxl': '89848fb8-549f-41bb-88cb-95d6597044a4',
 'playground_mixtral_8x7b': '8f4118ba-60a8-4e6b-8574-e38a4067a4a3',
 'playground_cuopt': '8f2fbd00-2633-41ce-ab4e-e5736d74bff7',
 'playground_clip': '8c21289c-0b18-446d-8838-011b7249c513',
 'playground_nemotron_steerlm_8b': '1423ff2f-d1c7-4061-82a7-9e8c67afd43a',
 'playground_fuyu_8b': '9f757064-657f-4c85-abd7-37a7a9b6ee11',
 'playground_sd_video': 'a529a395-a7a0-4708-b4df-eb5e41d5ff60',
 'playground_nvolveqa_40k': '091a03bb-7364-4087-8090-bd71e9277520',
 'playground_neva_22b': '8bf70738-59b9-4e5f-bc87-7ab4203be7a0',
 'playground_deplot': '3bc390c7-eeec-40f7-a64d-0c6a719985f7',
 'playground_llama2_13b': 'e0bb7fb9-5333-4a27-8534-c6288f921d3f',
 'playground_nemotron_qa_8b': '0c60f14d-46cb-465e-b994-227e1c3d5047',
 'playgrou

----

<br>

## Part 1: Summary of RAG Workflows

This notebook will explore several paradigms and derive reference code to help you approach some of the most common retrieval-augmented workflows. Specifically, the following sections will be covered (with the differences highlighted):

<br>

> ***Vector Store Workflow for Conversational Exchanges:***
- Generate semantic embedding for each new conversation.
- Add the message body to a vector store for retrieval.
- Query the vector store for relevant messages to fill in the LLM context.

<br>

> ***Modified Workflow for an Arbitrary Document:***
- **Divide the document into chunks and process them into useful messages.**
- Generate semantic embedding for each **new document chunk**.
- Add the **chunk bodies** to a vector store for retrieval.
- Query the vector store for relevant **chunks** to fill in the LLM context.
    - ***Optional:* Modify/synthesize results for better LLM results.**

<br>

> **Extended Workflow for a Directory of Arbitrary Documents:**
- Divide **each document** into chunks and process them into useful messages.
- Generate semantic embedding for each new document chunk.
- Add the chunk bodies to **a scalable vector database for fast retrieval**.
    - ***Optional*: Exploit hierarchical or metadata structures for larger systems.**
- Query the **vector database** for relevant chunks to fill in the LLM context.
    - *Optional:* Modify/synthesize results for better LLM results.

<br>

Some of the most important terminology surrounding RAG is covered in detail on the [**LlamaIndex Concepts page**](https://docs.llamaindex.ai/en/stable/getting_started/concepts.html), which itself is a great starting point for progressing towards the LlamaIndex loading and retrieving strategy. We highly recommend using it as a reference as you continue with this notebook and advise you to try out LlamaIndex after the course to consider the pros and cons firsthand!


<!-- > <img src="https://drive.google.com/uc?export=view&id=1cFbKbVvLLnFPs3yWCKIuzXkhBWh6nLQY" width=1200px/> -->
> <img src="https://dli-lms.s3.amazonaws.com/assets/s-fx-15-v1/imgs/data_connection_langchain.jpeg" width=1200px/>
>
> From [**Retrieval | LangChain**🦜️🔗](https://python.langchain.com/docs/modules/data_connection/)

----

<br>

## **Part 2:** RAG for Conversation History

In our previous explorations, we delved into the capabilities of document embedding models and used them to embed, store, and compare semantic vector representations of text. Though we could motivate how to efficiently extend this into vector store land manually, the true beauty of working with a standard API is its strong incorporation with other frameworks that can already do the heavy lifting for us!

<br>

### **Step 1**: Getting A Conversation

Consider a conversation crafted using Llama-13B between a chat agent and a blue bear named Beras. This dialogue, dense with details and potential diversions, provides a rich dataset for our study:


In [3]:
conversation = [  ## This conversation was generated partially by an AI system, and modified to exhibit desirable properties
    "[User]  Hello! My name is Beras, and I'm a big blue bear! Can you please tell me about the rocky mountains?",
    "[Agent] The Rocky Mountains are a beautiful and majestic range of mountains that stretch across North America",
    "[Beras] Wow, that sounds amazing! Ive never been to the Rocky Mountains before, but Ive heard many great things about them.",
    "[Agent] I hope you get to visit them someday, Beras! It would be a great adventure for you!"
    "[Beras] Thank you for the suggestion! Ill definitely keep it in mind for the future.",
    "[Agent] In the meantime, you can learn more about the Rocky Mountains by doing some research online or watching documentaries about them."
    "[Beras] I live in the arctic, so I'm not used to the warm climate there. I was just curious, ya know!",
    "[Agent] Absolutely! Lets continue the conversation and explore more about the Rocky Mountains and their significance!"
]

Using the manual embedding strategy from the previous notebook is still very viable, but we can also rest easy and let a **vector store** do all that work for us!

<br>

### **Step 2:** Constructing Our Vector Store Retriever

To streamline similarity queries on our conversation, we can employ a vector store to help keep track of passages for us! **Vector Stores**, or vector storage systems, abstract away most of the low-level details of the embedding/comparison strategies and provide a simple interface to load and compare vectors.


<!-- > <img src="https://drive.google.com/uc?export=view&id=1ZjwYbSZzsXK6ZP8O1-cY3BeRffV4oqzb" width=1000px/> -->
> <img src="https://dli-lms.s3.amazonaws.com/assets/s-fx-15-v1/imgs/vector_stores.jpeg" width=1200px/>
>
> From [**Vector Stores | LangChain**🦜️🔗](https://python.langchain.com/docs/modules/data_connection/vectorstores/)

<br>

In addition to simplifying the process from an API perspective, vector stores also implement connectors, integrations, and optimizations under the hood. In our case, we will start with the [**FAISS vector store**](https://python.langchain.com/docs/integrations/vectorstores/faiss), which integrates a LangChain-compatable Embedding model with the [**FAISS (Facebook AI Similarity Search)**](https://github.com/facebookresearch/faiss) library to make the process fast and scalable on our local machine!

**Specifically:**

1. We can feed our conversation into [**a FAISS vector store**](https://python.langchain.com/docs/integrations/vectorstores/faiss) via the `from_texts` constructor. This will take our conversational data and the embedding model to create a searchable index over our discussion.
2. This vector store can then be "interpreted" as a retriever, supporting the LangChain runnable API and returning documents retrieved via an input query.

The following shows how you can construct a FAISS vector store and reinterpret it as a retriever using the LangChain `vectorstore` API:

In [4]:
%%time
## ^^ This cell will be timed to see how long the conversation embedding takes
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain.vectorstores import FAISS

embedder = NVIDIAEmbeddings(model="nvolveqa_40k")

## Streamlined from_texts FAISS vectorstore construction from text list
convstore = FAISS.from_texts(conversation, embedding=embedder)
retriever = convstore.as_retriever()

CPU times: user 664 ms, sys: 364 ms, total: 1.03 s
Wall time: 1.64 s


The retriever can now be used like any other LangChain runnable to query the vector store for some relevant documents:

In [5]:
pprint(retriever.invoke("What is your name?"))

[
    Document(
        page_content='[Beras] Wow, that sounds amazing! Ive never been to the Rocky Mountains before, but Ive heard
many great things about them.'
    ),
    Document(
        page_content="[User]  Hello! My name is Beras, and I'm a big blue bear! Can you please tell me about the 
rocky mountains?"
    ),
    Document(
        page_content='[Agent] The Rocky Mountains are a beautiful and majestic range of mountains that stretch 
across North America'
    ),
    Document(
        page_content='[Agent] Absolutely! Lets continue the conversation and explore more about the Rocky Mountains
and their significance!'
    )
]

In [6]:
pprint(retriever.invoke("Where are the Rocky Mountains?"))

[
    Document(
        page_content='[Beras] Wow, that sounds amazing! Ive never been to the Rocky Mountains before, but Ive heard
many great things about them.'
    ),
    Document(
        page_content='[Agent] The Rocky Mountains are a beautiful and majestic range of mountains that stretch 
across North America'
    ),
    Document(
        page_content='[Agent] Absolutely! Lets continue the conversation and explore more about the Rocky Mountains
and their significance!'
    ),
    Document(
        page_content="[Agent] In the meantime, you can learn more about the Rocky Mountains by doing some research 
online or watching documentaries about them.[Beras] I live in the arctic, so I'm not used to the warm climate 
there. I was just curious, ya know!"
    )
]

As we can see, our retriever found a handful of semantically relevant documents from our query. You may notice that not all of the documents are useful or clear on their own. For example, a retrieval of *"Beras"* for *"your name"* may be problematic for the chatbot if provided out of context. Anticipating the potential problems and creating synergies between your LLM components can increase the likelihood of good RAG behavior, so keep an eye out for such pitfalls and opportunities.

<br>

### **Step 3:** Incorporating Conversation Retrieval Into Our Chain

Now that we have our loaded retriever component as a chain, we can incorporate it into our existing chat system as before. Specifically, we can start with an ***always-on RAG formulation*** where:
- **A retriever is always retrieving context by default**.
- **A generator is acting on the retrieved context**.

In [10]:
from langchain.document_transformers import LongContextReorder
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda
from langchain.schema.runnable.passthrough import RunnableAssign
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from functools import partial
from operator import itemgetter

########################################################################
## Utility Runnables/Methods
def RPrint(preface=""):
    """Simple passthrough "prints, then returns" chain"""
    def print_and_return(x, preface):
        print(f"{preface}{x}")
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

def docs2str(docs, title="Document"):
    """Useful utility for making chunks into context string. Optional, but useful"""
    out_str = ""
    for doc in docs:
        doc_name = getattr(doc, 'metadata', {}).get('Title', title)
        if doc_name:
            out_str += f"[Quote from {doc_name}] "
        out_str += getattr(doc, 'page_content', str(doc)) + "\n"
    return out_str

## Optional; Reorders longer documents to center of output text
long_reorder = RunnableLambda(LongContextReorder().transform_documents)
########################################################################

llm = ChatNVIDIA(model="mixtral_8x7b") | StrOutputParser()

context_prompt = ChatPromptTemplate.from_messages([
    ('system',
        "Answer the question using only the context"
        "\n\nQuestion: {question}\n\nContext: {context}" ## Double reinforcement
    ), ('user', "{question}"),
])

chain = (
    {
        'context': convstore.as_retriever() | long_reorder | docs2str,
        'question': (lambda x:x)
    }
    | context_prompt
    #| RPrint()
    | llm
    | StrOutputParser()
)

pprint(chain.invoke("What is my name?"))

Based on the context provided, your name is Beras.

Take a second to try out some more invocations and see how the new setup performs. Regardless of your model choice, the following questions should serve as interesting starting points.

In [11]:
pprint(chain.invoke("Where are the Rocky Mountains?"))

The Rocky Mountains stretch across North America. (according to the context provided)

In [12]:
pprint(chain.invoke("Where are the Rocky Mountains? Are they close to California?"))

Exception: [500] Internal Server Error
string_value:"[StatusCode.DEADLINE_EXCEEDED] Deadline Exceeded"

In [ ]:
pprint(chain.invoke(
    "Where are the Rocky Mountains? Please include"
    " the author's reasoning, but provide more information!"
))

In [ ]:
pprint(chain.invoke("How far away is Beras from the Rocky Mountains?"))

<br>

You might notice some decent performance with this always-on retrieval node in the loop since the actual context being fed into the LLM remains relatively small. It's important to experiment with factors like embedding sizes, context limits, and model options to see what kinds of behavior you can expect and which efforts are worth taking to improve performance.

<br>

### **Step 4:** Automatic Conversation Storage

Now that we see how our vector store memory unit should function, we can perform one last integration to allow our conversation to add new entries to our conversation: a runnable that calls the `add_texts` method for us to update the store state.


In [13]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

########################################################################
## Reset knowledge base and define what it means to add more messages.
convstore = FAISS.from_texts(conversation, embedding=embedder)

def save_memory_and_get_output(d, vstore):
    """Accepts 'input'/'output' dictionary and saves to convstore"""
    vstore.add_texts([f"User said {d.get('input')}", f"Agent said {d.get('output')}"])
    return d.get('output')

########################################################################

llm = ChatNVIDIA(model="mixtral_8x7b") | StrOutputParser()

chat_prompt = ChatPromptTemplate.from_messages([("system",
    "A user has asked a question: {input}\n\n Context: \n{context}\n\n"
    "Please continue the conversation by responding! Keep it brief and conversational!"
), ('user', '{input}')])

conv_chain = (
    {
        'context': convstore.as_retriever() | long_reorder | docs2str,
        'input': (lambda x:x)
    }
    | RunnableAssign({'output' : chat_prompt | llm})
    | partial(save_memory_and_get_output, vstore=convstore)
)

pprint(conv_chain.invoke("I'm glad you agree! I can't wait to get some ice cream there! It's such a good food!"))
print()
pprint(conv_chain.invoke("Can you guess what my favorite food is?"))
print()
pprint(conv_chain.invoke("Actually, it's honey! Not sure where you got that idea?"))
print()
pprint(conv_chain.invoke("I see! Fair enough! Do you know my favorite food now?"))

That sounds like a great plan! I'm sure there are plenty of delicious ice cream shops in the Rocky Mountains. Have 
you decided which specific location you'd like to visit? We can also continue to chat about the Rocky Mountains and
their history if you'd like.

Based on your excitement about ice cream, I'm guessing that could be your favorite food! Do you have a preferred 
flavor or topping?

Exception: [500] Internal Server Error
string_value:"[StatusCode.DEADLINE_EXCEEDED] Deadline Exceeded"

Unlike the more automatic full-text or rule-based approaches to injecting context into the LLM, this approach ensures some amount of consolidation which can keep the context length from getting out of hand. It's still not a full-proof strategy on its own, but it's a stark improvement for unstructured conversations (and doesn't even require a strong instruction-tuned model to perform slot-filling).

----

<br>

## **Part 3 [Exercise]:** RAG For Document Chunk Retrieval

Given our prior exploration of document loading, the idea that data chunks can be embedded and searched through probably isn't surprising. With that said, it is definitely worth going over since applying RAG with documents is a double-edged sword; it may **seem** to work well out of the box but requires some extra care when optimizing it for truly reliable performance. It also provides an excellent opportunity to review some fundamental LCEL skills, so let's see what we can do!

<br>

### **Exercise:**

In the previous example, you may recall that we pulled in some relatively small papers with the help of [`ArxivLoader`](https://python.langchain.com/docs/integrations/document_loaders/arxiv) using the following syntax:

```python
from langchain.document_loaders import ArxivLoader

docs = [
    ArxivLoader(query="2205.00445").load(),  ## MRKL
    ArxivLoader(query="2210.03629").load(),  ## ReAct
]
```

Given all that you've learned so far, choose a selection of papers that you would like to use and develop a chatbot that can talk about them!

<br>

Though this is a pretty big task, a walkthrough of ***most*** of the process will be provided below. By the end of the walkthrough, many of the necessary puzzle pieces will be provided, and your real task will be to integrate them together for the final `retrieval_chain`. When you're done, get ready to re-integrate the chain (or a flavor of your choice) in the last notebook as part of the evaluation exercise!


<br>

### **Task 1**: Loading And Chunking Your Documents

The following code block gives you some default papers to load in for your RAG chain. Feel free to select more papers as desired, but note that longer documents will take longer to process. A few simplifying assumptions and additional processing steps are included to help you improve your naive RAG performance:

- Documents are cut off prior to the "References" section if one exists. This will keep our system from considering the citations and appendix sections, which tend to be long and distracting.

- A chunk that lists the available documents is inserted to provide a high-level view of all available documents in a single chunk. If your pipeline does not provide metadata on each retrieval, this is a useful component and can even be listed among a list of higher-priority pieces if appropriate.

- Additionally, the metadata entries are also inserted to provide general information. Ideally, there would also be some synthetic chunks that merge the metadata into interesting cross-document chunks.

**NOTE:** ***For the sake of the assessment, please include at least one paper that is less than one month old!***


In [14]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import ArxivLoader

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=100,
    separators=["\n\n", "\n", ".", ";", ",", " ", ""],
)

## TODO: Please pick some papers and add them to the list as you'd like
## NOTE: To re-use for the final assessment, make sure at least one paper is < 1 month old
print("Loading Documents")
docs = [
    ArxivLoader(query="1706.03762").load(),  ## Attention Is All You Need Paper
    ArxivLoader(query="1810.04805").load(),  ## BERT Paper
    ArxivLoader(query="2005.11401").load(),  ## RAG Paper
    ArxivLoader(query="2205.00445").load(),  ## MRKL Paper
    ArxivLoader(query="2310.06825").load(),  ## Mistral Paper
    ArxivLoader(query="2306.05685").load(),  ## LLM-as-a-Judge
    ## Some longer papers
    ArxivLoader(query="2210.03629").load(),  ## ReAct Paper
    ArxivLoader(query="2112.10752").load(),  ## Latent Stable Diffusion Paper
    ArxivLoader(query="2103.00020").load(),  ## CLIP Paper
    ## TODO: Feel free to add more
]

## Cut the paper short if references is included.
## This is a standard string in papers.
for doc in docs:
    content = doc[0].page_content
    if "References" in content:
        doc[0].page_content = content[:content.index("References")]

## Split the documents and also filter out stubs (overly short chunks)
print("Chunking Documents")
docs_chunks = [text_splitter.split_documents(doc) for doc in docs]
docs_chunks = [[c for c in dchunks if len(c.page_content) > 200] for dchunks in docs_chunks]

## Make some custom Chunks to give big-picture details
doc_string = "Available Documents:"
doc_metadata = []
for chunks in docs_chunks:
    metadata = getattr(chunks[0], 'metadata', {})
    doc_string += "\n - " + metadata.get('Title')
    doc_metadata += [str(metadata)]

extra_chunks = [doc_string] + doc_metadata

## Printing out some summary information for reference
pprint(doc_string, '\n')
for i, chunks in enumerate(docs_chunks):
    print(f"Document {i}")
    print(f" - Metadata: {chunks[0].metadata}")
    print(f" - # Chunks: {len(chunks)}")
    print()

Loading Documents
Chunking Documents


Available Documents:
 - Attention Is All You Need
 - BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding
 - Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks
 - MRKL Systems: A modular, neuro-symbolic architecture that combines large language models, external knowledge 
sources and discrete reasoning
 - Mistral 7B
 - Judging LLM-as-a-Judge with MT-Bench and Chatbot Arena
 - ReAct: Synergizing Reasoning and Acting in Language Models
 - High-Resolution Image Synthesis with Latent Diffusion Models
 - Learning Transferable Visual Models From Natural Language Supervision 

Document 0
 - Metadata: {'Published': '2023-08-02', 'Title': 'Attention Is All You Need', 'Authors': 'Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin', 'Summary': 'The dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks in an encoder-decoder configuration. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer, based\nsolely on attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two machine translation tasks show these models to be\nsuperior in quality while being more parallelizable and requiring significantly\nless time to train. Our model achieves 28.4 BLEU on the WMT 2014\nEnglish-to-German translation task, improving over the existing best results,\nincluding ensembles by over 2 BLEU. On the WMT 2014 English-to-Frenc

<br>

### **Task 2**: Construct Your Document Vector Stores

Now that we have all of the components, we can go ahead and create indices surrounding them:

In [15]:
%%time
## ^^ This cell will output a time
from faiss import IndexFlatL2
from langchain_community.docstore.in_memory import InMemoryDocstore

from langchain_core.prompts import ChatPromptTemplate

embedder = NVIDIAEmbeddings(model="nvolveqa_40k", model_type=None)

## Construct series of document vector stores
print("Constructing Vector Stores")
vecstores = [FAISS.from_texts(extra_chunks, embedder)]
vecstores += [FAISS.from_documents(doc_chunks, embedder) for doc_chunks in docs_chunks]

Constructing Vector Stores
CPU times: user 1.64 s, sys: 13.5 ms, total: 1.65 s
Wall time: 36 s


<br>

From there, we can combine our indices into a single one using the following utility:

In [16]:
embed_dims = len(embedder.embed_query("test"))
def default_FAISS():
    '''Useful utility for making an empty FAISS vectorstore'''
    return FAISS(
        embedding_function=embedder,
        index=IndexFlatL2(embed_dims),
        docstore=InMemoryDocstore(),
        index_to_docstore_id={},
        normalize_L2=False
    )

def aggregate_vstores(vectorstores):
    ## Initialize an empty FAISS Index and merge others into it
    ## We'll use default_faiss for simplicity, though it's tied to your embedder by reference
    agg_vstore = default_FAISS()
    for vstore in vectorstores:
        agg_vstore.merge_from(vstore)
    return agg_vstore

if 'docstore' not in globals():
    ## Unintuitive optimization; merge_from seems to optimize constituent vector stores away
    docstore = aggregate_vstores(vecstores)

print(f"Constructed aggregate docstore with {len(docstore.docstore._dict)} chunks")

Constructed aggregate docstore with 542 chunks


<br>

### **Task 3: [Exercise]** Implement Your RAG Chain

Finally, all the puzzle pieces are in place to implement the RAG pipeline! As a review, we now have:

- A way to construct a from-scratch vector store for conversational memory (and a way to initialize an empty one with `default_FAISS()`)

- A vector store pre-loaded with useful document information from our `ArxivLoader` utility (stored in `docstore`).

With the help of a couple more utilities, you're finally ready to integrate your chain! A few additional convenience utilities are provided (`doc2str` and the now-common `RPrint`) but are optional to use. Additionally, some starter prompts and structures are also defined.

> **Given all of this:** Please implement the `retrieval_chain`.

In [19]:
from langchain.document_transformers import LongContextReorder
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.passthrough import RunnableAssign
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

import gradio as gr
from functools import partial
from operator import itemgetter


llm = ChatNVIDIA(model="mixtral_8x7b") | StrOutputParser()
convstore = default_FAISS()

def save_memory_and_get_output(d, vstore):
    """Accepts 'input'/'output' dictionary and saves to convstore"""
    vstore.add_texts([
        f"User previously responded with {d.get('input')}",
        f"Agent previously responded with {d.get('output')}"
    ])
    return d.get('output')

initial_msg = (
    "Hello! I am a document chat agent here to help the user!"
    f" I have access to the following documents: {doc_string}\n\nHow can I help you?"
)

chat_prompt = ChatPromptTemplate.from_messages([("system",
    "You are a document chatbot. Help the user as they ask questions about documents."
    " User messaged just asked: {input}\n\n"
    " From this, we have retrieved the following potentially-useful info: "
    " Conversation History Retrieval:\n{history}\n\n"
    " Document Retrieval:\n{context}\n\n"
    " (Answer only from retrieval. Only cite sources that are used. Make your response conversational.)"
), ('user', '{input}')])

################################################################################################
## BEGIN TODO: Implement the retrieval chain to make your system work!

retrieval_chain = (
    {'input' : (lambda x: x)}
    ## TODO: Make sure to retrieve history & context from convstore & docstore, respectively.
    ## HINT: Our solution uses RunnableAssign, itemgetter, long_reorder, and docs2str
    |RunnableAssign({'history' : itemgetter('input')| convstore.as_retriever() | long_reorder | docs2str})
    |RunnableAssign({'context' : itemgetter('input')| convstore.as_retriever() | long_reorder | docs2str})
    |RPrint()
    
    
)

## END TODO
################################################################################################

stream_chain = chat_prompt | llm

def chat_gen(message, history=[], return_buffer=True):
    buffer = ""
    ## First perform the retrieval based on the input message
    retrieval = retrieval_chain.invoke(message)
    line_buffer = ""

    ## Then, stream the results of the stream_chain
    for token in stream_chain.stream(retrieval):
        buffer += token
        ## If you're using standard print, keep line from getting too long
        if not return_buffer:
            line_buffer += token
            if "\n" in line_buffer:
                line_buffer = ""
            if ((len(line_buffer)>84 and token and token[0] == " ") or len(line_buffer)>100):
                line_buffer = ""
                yield "\n"
                token = "  " + token.lstrip()
        yield buffer if return_buffer else token

    ## Lastly, save the chat exchange to the conversation memory buffer
    save_memory_and_get_output({'input':  message, 'output': buffer}, convstore)


## Start of Agent Event Loop
test_question = "Tell me about RAG!"  ## <- modify as desired

## Before you launch your gradio interface, make sure your thing works
for response in chat_gen(test_question, return_buffer=False):
    print(response, end='')

{'input': 'Tell me about RAG!', 'history': '', 'context': ''}
RAG is an acronym that can stand for a few different things, but in the context of
  documents or data visualization, it usually refers to "Red Amber Green" status indicators.
  These indicators are often used in dashboards or reports to quickly convey the status of
  a particular metric or key performance indicator (KPI).

Here's a breakdown of what each color might represent:

* Red: This indicates a problem or issue that needs attention. It might mean that a KPI
  is not meeting its target or that there is some other negative trend occurring.
* Amber (or Yellow): This indicates a caution or warning. It might mean that a KPI is
  close to missing its target or that there is a potential issue that warrants monitoring.
* Green: This indicates that everything is on track and no issues have been
  identified.

By using these simple color-coded indicators, it's easy to quickly understand the
  status of various metrics and KPIs

### **Task 4:** Interact With Your Gradio Chatbot

In [ ]:
chatbot = gr.Chatbot(value = [[None, initial_msg]])
demo = gr.ChatInterface(chat_gen, chatbot=chatbot).queue()

try:
    demo.launch(debug=True, share=True, show_api=False)
    demo.close()
except Exception as e:
    demo.close()
    print(e)
    raise e

<br>

----

<br>

## **Part 4:** Saving Your Index For Evaluation

After you've implemented your RAG chain, please save your accumulated vector store as shown [in the official documentation](https://python.langchain.com/docs/integrations/vectorstores/faiss#saving-and-loading). You'll have a chance to use it again for your final assessment!

In [20]:
## Save and compress your index
docstore.save_local("docstore_index")
!tar czvf docstore_index.tgz docstore_index

!rm -rf docstore_index

docstore_index/
docstore_index/index.pkl
docstore_index/index.faiss


If everything was properly saved, the following line can be invoked to pull the index from the compressed `tgz` file (assuming the pip requirements are installed). After you have confirmed that the cell can pull in your index, download `docstore_index.tgz` for use in the last notebook!

In [21]:
## Assuming you haven't imported these yet
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain_community.vectorstores import FAISS

embedder = NVIDIAEmbeddings(model="nvolveqa_40k")
!tar xzvf docstore_index.tgz
new_db = FAISS.load_local("docstore_index", embedder)
docs = new_db.similarity_search("Testing the index")
print(docs[0].page_content[:1000])

docstore_index/
docstore_index/index.pkl
docstore_index/index.faiss
������������
����������������������
������������������������� �
��������� ���������������� �
���������������������� �
����������������������� �
�������������������������� �
������������������������� �
����������������������� �
���������������������� �
������
�������������������������� �
�����
��������������
��������������������������� �
�������������������������� ���
����������������
����������������������� ��
���������������������������� �
����������������
����������������������� �
���������� ��
��������������������� �
������������������������� �
����������
������������������
�������������������������
����������������������������������������������������� �
����������������������������������������������������
���������������������������
���������������������������������������������������������
�
����������������������������������������������������������
�
���������������������� ����������
������������������������������

-----

<br>

## **Part 5:** Wrap-Up

Congratulations! Assuming your RAG chain is all good, you're now ready to move on to the **RAG Evaluation [Assessment]** section!

### <font color="#76b900">**Great Job!**</font>

### **Next Steps:**
1. **[Optional]** Revisit the **"Questions To Think About" Section** at the top of the notebook and think about some possible answers.
2. Continue on to the next video, which will discuss **RAG Evaluation**.
3. After the video, continue on to the corresponding notebook on **RAG Evaluation [Assessment]**.

---

<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>